In [10]:
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'custom_features'

In [4]:
for i in range(1,6): #iteration on the sets 
    #Load the npz files of training, testing and validation sets for each iteration
    # load training
    loaded_data_train = np.load(f'../Feature_Selection/training_features_{i}.npz')
    x_train = loaded_data_train['matrix']
    y_train = loaded_data_train['target']
    
    # load test
    loaded_data_test = np.load(f'../Feature_Selection/testing_features_{i}.npz')
    x_test = loaded_data_test['matrix']
    y_test = loaded_data_test['target']
    
    # load validation
    loaded_data_validation = np.load(f'../Feature_Selection/validation_features_{i}.npz')
    x_validation = loaded_data_validation['matrix']
    y_validation = loaded_data_validation['target']

    training_set_conc = np.concatenate((x_train, x_test), axis=0)
    training_class_conc = np.concatenate((y_train, y_test), axis=0)
    